In [1]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import display
from fuzzywuzzy import fuzz, process
from ITUtils import country_conflicts_finder  # Assuming you have this util function
from pandarallel import pandarallel

# show all columns
pd.set_option('display.max_rows', None)

In [2]:
# pick a country and import the conflicts
adm = 'G'
file = 'expanded_combined_tables_conflicts_lettersatnames.csv'
filepath = os.path.join('.', 'adm_conflicts', adm, file)
df = pd.read_csv(filepath, low_memory=False)
display(df.head())
print(df.columns)

,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.long_nom,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,...,TPA1.2055.475-2055.725_R,TPA1.401.89525-401.90475_R,TPA1.2237.0-2238.0_E,TPA1.2065.575-2065.825_R,TPA1.401.9501-401.9699_E,TPA1.401.8901-401.9099_R,TPA1.401.95525-401.96475_E,TPA1.401.8901-401.9099_E,TPA1.401.89525-401.90475_E,TPA1.2202.4-2203.4_E
0,118545036,NaN,G,NaN,SSG-CSL,NaN,9.1/IA,30.05.2018,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,118545036,NaN,G,NaN,SSG-CSL,NaN,9.1/IA,30.05.2018,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118545036,NaN,G,NaN,SSG-CSL,NaN,9.1/IA,30.05.2018,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,118545036,NaN,G,NaN,SSG-CSL,NaN,9.1/IA,30.05.2018,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,118545036,NaN,G,NaN,SSG-CSL,NaN,9.1/IA,30.05.2018,50,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index(['com_el.ntc_id', ' com_el.tgt_ntc_id', ' com_el.adm',
       ' com_el.ntwk_org', ' com_el.sat_name', ' com_el.long_nom',
       ' com_el.prov', ' com_el.d_rcv', ' com_el.st_cur', ' orbit.orb_id',
       ' orbit.nbr_sat_pl', ' orbit.apog_km', ' orbit.perig_km',
       ' orbit.op_ht_km', ' s_beam.emi_rcp', ' s_beam.beam_name',
       ' grp.grp_id', ' grp.freq_min', ' grp.freq_max', ' grp.bdwdth',
       ' grp.d_inuse', ' grp.d_reg_limit', ' grp.d_prot_eff', ' grp.f_biu',
       ' emiss.seq_no', ' emiss.pwr_ds_max', ' emiss.design_emi',
       ' carrier_fr.freq_carr', ' channel.bandwidth', ' channel.freq_min',
       ' channel.freq_max', 'tpaconflicts', 'percentoverlap',
       'TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R',
       'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R',
       'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R',
       'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R',
       'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E',

In [3]:
# funcition to extract the table
# Initialize pandarallel
num_logical_processors = os.cpu_count()
pandarallel.initialize(nb_workers=num_logical_processors, progress_bar=True)


def summary_table(df):
    # columns to keep
    cols = [
        ' com_el.sat_name',
        ' s_beam.beam_name',
        ' carrier_fr.freq_carr',
        ' channel.bandwidth',
        'tpaconflicts',
        'percentoverlap'
    ]
    lookupnames = [
        'UHFUP fc=401.96 bw=9K5',
        'UHFUP fc=401.96 bw=19K8',
        'SUP fc=2055.6 bw=250K',
        'UHFUP fc=401.90 bw=9K5',
        'SDN fc=2237.5 bw=1M',
        'SUP fc=2065.7 bw=250k',
        'UHFDN fc=401.96 bw=19K8',
        'UHFUP fc=401.90 bw=19K8',
        'UHFDN fc=401.96 bw=9K5',
        'UHFDN fc=401.90 bw=19K8',
        'UHFDN fc=401.90 bw=9K5',
        'SDN fc=2202.9 bw=1M'
    ]
    # get the columns
    df1 = df[cols].copy()
    if df1.empty:
        print("❗ DataFrame is empty")
        return pd.DataFrame(columns=[
            'Sat Name', 'Beam Name', 'Carrier Freq. (fc MHz)', 'Bandwidth (MHz)', 'Overlaps'
        ])
    # add a literal column 
    df1.loc[:, 'Overlaps'] = None

    def functiontoapply(row):
        con = str(row['tpaconflicts']).split(':')
        confidx = [] 
        for c in con[:-1]:
            confidx.append(int(c))
        perc = str(row['percentoverlap']).split(':')
        overlap = []
        for i in range(len(confidx)):
            over = str(lookupnames[confidx[i]]) + ' ' + str(perc[i]) + '% overlap'
            overlap.append(over)
        row['Overlaps'] = ' : '.join(overlap)
        if row['Overlaps'] == '':
            row['Overlaps'] = 'No Conflicts Found'
        return row

    # apply in parallel to all rows
    if len(df1) > 24:
        df1 = df1.parallel_apply(functiontoapply, axis=1)
    else:
        df1 = df1.apply(functiontoapply, axis=1)
    # divide the bandwidth column by 1e6
    df1[' channel.bandwidth'] = df1[' channel.bandwidth'].apply(lambda x: float(x) / 1e6)

    
    # drop columns 
    df1 = df1.drop(['tpaconflicts', 'percentoverlap'], axis=1)
    # rename columns
    df1 = df1.rename(columns={
        ' com_el.sat_name': 'Sat Name',
        ' s_beam.beam_name': 'Beam Name',
        ' carrier_fr.freq_carr': 'Carrier Freq. (fc MHz)',
        ' channel.bandwidth': 'Bandwidth (MHz)', })  # this is calculated from the emission designator code e.g. 19k8XXX
    # drop duplicates
    df1 = df1.drop_duplicates()
    return df1



INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
# apply to dataframe
table = summary_table(df)

# export 
outpath = os.path.join('.', 'adm_conflicts', adm, 'conflicts_summary.csv')
table.to_csv(outpath, index=False)
print('File saved to ', outpath)

File saved to  ./adm_conflicts/G/conflicts_summary.csv


In [5]:
# iterate for every administration 
# === CONFIG ===
tpafile = './databases/TPAtable.csv'
tablesfolder = 'countriestables'
outfolder = 'adm_conflicts'
countrieslistfile = 'countrieslist.csv'
# Load country codes
with open(countrieslistfile, 'r') as f:
    countries = f.read().strip().split(', ')
# === PROCESS EACH COUNTRY ===
for ccode in countries:
    print(f"\n=== Processing {ccode} ===")
    # outfolder (must already exist)
    country_outfolder = os.path.join(outfolder, ccode)
    adm = ccode
    file = 'expanded_combined_tables_conflicts_lettersatnames.csv'
    filepath = os.path.join('.', 'adm_conflicts', adm, file)
    df = pd.read_csv(filepath, low_memory=False)
    # display(df.head())
    # print(df.columns)
    # execute 
    table = summary_table(df)
        # export 
    outpath = os.path.join('.', 'adm_conflicts', adm, 'conflicts_summary.csv')
    table.to_csv(outpath, index=False)
    print('File saved to ', outpath)
    
    # do the same for the non filtered satellite names
    file = 'expanded_combined_tables_conflicts_othersatnames.csv'
    filepath = os.path.join('.', 'adm_conflicts', adm, file)
    df = pd.read_csv(filepath, low_memory=False)
    # display(df.head())
    # print(df.columns)
    # execute 
    table = summary_table(df)
        # export 
    outpath = os.path.join('.', 'adm_conflicts', adm, 'conflicts_summary_othersatnames.csv')
    table.to_csv(outpath, index=False)
    print('File saved to ', outpath)



=== Processing AFS ===
File saved to  ./adm_conflicts/AFS/conflicts_summary.csv
File saved to  ./adm_conflicts/AFS/conflicts_summary_othersatnames.csv

=== Processing ARS ===


File saved to  ./adm_conflicts/ARS/conflicts_summary.csv


File saved to  ./adm_conflicts/ARS/conflicts_summary_othersatnames.csv

=== Processing AUS ===


File saved to  ./adm_conflicts/AUS/conflicts_summary.csv


File saved to  ./adm_conflicts/AUS/conflicts_summary_othersatnames.csv

=== Processing CAN ===


...

File saved to  ./adm_conflicts/PAK/conflicts_summary_othersatnames.csv

=== Processing POR ===
File saved to  ./adm_conflicts/POR/conflicts_summary.csv
File saved to  ./adm_conflicts/POR/conflicts_summary_othersatnames.csv

=== Processing RUS ===


File saved to  ./adm_conflicts/RUS/conflicts_summary.csv
❗ DataFrame is empty
File saved to  ./adm_conflicts/RUS/conflicts_summary_othersatnames.csv

=== Processing SLM ===
❗ DataFrame is empty
File saved to  ./adm_conflicts/SLM/conflicts_summary.csv


File saved to  ./adm_conflicts/SLM/conflicts_summary_othersatnames.csv

=== Processing THA ===


File saved to  ./adm_conflicts/THA/conflicts_summary.csv
❗ DataFrame is empty
File saved to  ./adm_conflicts/THA/conflicts_summary_othersatnames.csv

=== Processing USA ===
❗ DataFrame is empty
File saved to  ./adm_conflicts/USA/conflicts_summary.csv


File saved to  ./adm_conflicts/USA/conflicts_summary_othersatnames.csv
